## scala中的classOf、isInstanceOf、asInstanceOf
总而言之，我们把`classOf[T]`看成Java里的`T.class`, `obj.isInstanceOf[T]`看成 `obj instanceof T`, `obj.asInstanceOf[T]`看成`(T)obj`就对了。

如果对象是 null，则isInstanceOf 一定返回 false， asInstanceOf 一定返回 null；

**Scala中getClass 和 classOf**  
* p.getClass是对象的方法，classOf[T]应该算是关键字吧  
* isInstanceOf 只能判断出对象是否为指定类以及其子类的对象，而不能精确的判断出，对象就是指定类的对象；  
* 如果要求精确地判断出对象就是指定类的对象，那么就只能使用 getClass 和 classOf 了；  
* p.getClass 可以精确地获取对象的类，classOf[T]可以精确的获取类，然后使用 == 操作符即可判断；  

同样也要注意类型擦除造成的问题:
```
由于Scala像Java一样泛型存在类型擦除的原因，List(1).isInstanceOf[List[String]]及相当于List(1).isInstanceOf[List[_]], List(1)是List的实例。

还是由于泛型存在类型擦除的原因，1.asInstanceOf[String]在运行时会抛出ClassCastException异常，而List(1).asInstanceOf[List[String]]将不会。
```

## scala中对象的相等
总结起来就是这样：

* **equals方法和java的equals一样**  
* **==和equals一样(比较的是值)，唯一不同的是==的左边可以使null而不会抛出空指针异常，所以在scala中就别用equals了**  
* **eq和ne完全等同于java的==和!=**  
* **数组相等要用sameElements方法，它是可变的原始 Java 数组,与Scala 库中我们习惯使用的集合类型有着不同的方法。scala集合可以用==比较相等**  

很 难 准 确 地 为 实 例 实 现 一 个 可 靠 的 相 等 测 试。Joshua Block 的 畅 销 书 Effective Java (Addison-Wesley 出 版 社 出 版 ) 以 及 关 于 AnyRef.eq 的 Scaladoc 页面都描述了相等测试需要满足的要求。

Martin Odersky、Lex Spoon 和 Bill Venners 共同撰写了一篇关于 equals 和 hashCode 方法的非常棒的文章“如何用 Java 语言编写相等方法”(“How to Write an Equality Method in Java”,http://www.artima.com/lejava/articles/equality.html)。回想一下,在 case 类中,这些方法是自动创建的。

事实上,我从来没有写过我自己的 equals 和 hashCode 方法。我觉得,对于任何要使用的对象,如果可能需要测试相等性或需要作为 Map 的键(此时会调用 hashCode )的话,它们都应该定义为 case 类!

接下来,我们学习用于测试相等的不同方法。

### equals 方法
我们将用一个 case 类来展示不同的相等方法是如何工作的:
```scala
case class Person(firstName: String, lastName: String, age: Int)

val p1a = Person("Dean", "Wampler", 29)
val p1b = Person("Dean", "Wampler", 29)
val p2 = Person("Buck", "Trends", 30)
```

equals 方法用于测试值的相等,也就是说,如果 obj1 和 obj2 有相同的值, obj1 equalsobj2 为 true。 obj1 和 obj2 不需要指向同一个实例:
```scala
p1a equals p1a      // = true
p1a equals p1b      // = true
p1a equals p2       // = false
p1a equals null     // = false
null equals p1a     // 抛出 java.lang.NullPointerException异常
null equals null    // 抛出 java.lang.NullPointerException异常
```

所以, equals 的行为类似 Java 里的 equals 方法和 Ruby 里的 eql? 方法。

### == 和 != 方法
== 在很多语言中是一个操作符,但在 Scala 中是一个方法。在 Scala 2.10 中, == 在 Any 中被定义为 final ,用来代表 equals 。在 2.11 版本中改变了实现,但行为保持不变:
```scala
p1a == p1a      // = true
p1a == p1b      // = true
p1a == p2       // = false
```

所以, == 的行为与 equals 完全一样,即只测试值是否相等。当 null 在 == 左边时是个例外:
```scala
p1a == null      // = false
null == p1a      // = false
null == null     // = true (编译警告,这永远为true)
```

正如你预期的那样, != 表示不相等,与 !(obj1 == obj2) 等价:
```scala
p1a != p1a      // = false
p1a != p1b      // = false
p1a != p2       // = true
p1a != null     // = true
null != p1a     // = true
null != null    // = false (编译警告,这永远为false)
```
**在 Java、C++ 和 C# 中,== 操 作 符 测 试 的 是 引 用, 而 不 是 值。 与 此 相 反,Scala 的 == 测试的是值的相等性。**

### eq 和 ne 方法
eq 方法用于测试引用的相等性。也就是说,如果 obj1 和 obj2 指向内存中的同一个位置,obj1 eq obj2 就为 true。这两个方法只对 AnyRef 类型有定义:
```scala
p1a eq p1a      // = true
p1a eq p1b      // = false
p1a eq p2       // = false
p1a eq null     // = false
null eq p1a     // = false
null eq null    // = true (编译警告,这永远为true)
```

正如编译器为 null == null 提出警告, null eq null 也得到了相同的警告。

所以, eq 的行为就像 Java、C++ 和 C# 中的 == 操作符一样。

ne 方法与 eq 的相反,也就是说它与 !(obj1 eq obj2) 等价:

```scala
p1a ne p1a      // = false
p1a ne p1b      // = true
p1a ne p2       // = true
p1a ne null     // = true
null ne p1a     // = true
null ne null    // = false (编译警告,这永远为false)
```

### 数组相等和 sameElements 方法
比较两个数组,在 Scala 中并不能得出我们认为的显而易见的结果:
```scala
Array(1, 2) == Array(1, 2)    // = false
```
这令人惊讶。值得庆幸的是,有一个简单的解决方案,就是 sameElements 方法:
```scala
Array(1, 2) sameElements Array(1, 2)    // = true
```

实际上,我们最好要记住, Array 是我们熟知和喜爱的,它是可变的原始 Java 数组,与Scala 库中我们习惯使用的集合类型有着不同的方法。

所以,如果你试图比较数组,考虑一下用序列来比较是否会更好。(不使用序列来代替的一个理由是,你有时真的需要数组相对于序列的优势性能。)

与数组相反,序列(比如 List )的相等性的行为就符合你的期望:
```scala
List(1, 2) == List(1, 2)              // = true
List(1, 2) sameElements List(1, 2)    // = true
```

## Scala 单例对象
普通的类：

In [1]:
class Upper {
  def upper(strings: String*): Seq[String] = {
    strings.map((s:String) => s.toUpperCase())
  }
}

val up = new Upper
println(up.upper("Hello", "World!"))

ArrayBuffer(HELLO, WORLD!)


defined class Upper
up: Upper = $sess.cmd0Wrapper$Helper$Upper@1b148944

使用单例对象：

In [2]:
object Upper {
  def upper(strings: String*) = strings.map(_.toUpperCase())
}

println(Upper.upper("Hello", "World!"))

ArrayBuffer(HELLO, WORLD!)


defined object Upper

这段代码同样实现了相同的功能，但使用的字符却少了三分之一。当然是使用了简略的语法。

在第一行中，Upper 被声明为单例对象，Scala 将单例模式视为本语言的第一等级成员。尽管我们声明了一个类，不过Scala 运行时只会创建Upper 的一个实例。也就是说，你无法通过new 创建Upper 对象。就好像Java 使用静态类型一样，其他语言使用类成员（classlevelmember），Scala 则使用对象进行处理。由于Upper 中并不包含状态信息，所以我们此处的确不需要多个实例，使用单例便能满足需求。

单例模式具有一些弊端，也因此常被指责。例如在那些需要将对象值进行double 的单元测试中，如果使用了单例对象，便很难替换测试值。而且如果对一个实例执行所有的计算，会引发线程安全和性能的问题。不过正如静态方法或静态值有时适用于Java 这样的语言一样，单例有时候在Scala 中也是适用的。上述示例便是一个证明，**由于无须维护状态而且对象也不需要与外界交互，单例模式适用于上述示例。因此，使用Upper 对象时我们没有必要考虑测试双倍值的问题，也没有必要担心线程安全。**

Scala 为什么不支持静态类型呢？与那些允许静态成员（或类似结构）的语言相比，Scala 更信奉万物皆应为对象。相较于混入了静态成员和实例成员的语言，采用对象结构的Scala 更坚定地贯彻了这一方针。回想一下，Java的静态方法和静态域并未绑定到类型的实际实例中，而Scala 的对象则是某一类型的单例。

## 为什么Scala不支持三元表达式
Scala 中的 if 语句和几乎所有的其他语句都是具有返回值的表达式。因此我们能像下面展示的代码那样,将 if 表达式的结果值赋给其他变量。
```scala
val configFile = new java.io.File("somefile.txt")

val configFilePath = if (configFile.exists()) {
  configFile.getAbsolutePath()
} else {
  configFile.createNewFile()
  configFile.getAbsolutePath()
}
```

**if 语句返回值的类型也被称为所有条件分支的最小上界类型,也就是与每条 each 子句可能返回值类型最接近的父类型。**

Scala 中的 if 语句是一类表示式,像 predicate ? trueHandler() : falseHandler() 这种三元表达式对于 Scala 来说是多余的, 因此 Scala 并不支持三元表达式。

python也不支持三元表达式，类似的语法：
```python
y = True if 100 > 0 else False
```
scala中语法(if的括号不能省略)：
```scala
val y = if(100 > 0) true else false
```

## 为什么Scala表示参数化类型 使用方括号而不是尖括号
参数化类型，就好象Java 中的泛型类型。请注意，Scala 使用方括号（[…]）表示参数类型，而Java 使用角括号（<…>）。

Scala 的标识符，如方法名和变量名，中允许出现尖括号，例如定义“小于”方法时，该方法常被命名为<，这在Scala 语言中是允许的，而Java 则不允许标识符中出现这样的字符。因此，为了避免出现歧义，Scala 使用方括号而不是尖括号表示参数化类型，并且不允许在标识符中使用方括号。

## Scala 方法的定义为什么使用等号
Scala 方法的定义体出现在等号（=）之后。为什么使用等号呢？而不像Java 那样，使用花括号表示方法体呢？

避免歧义是原因之一。当你在代码中省略分号时，Scala 能够推断出来。在大多数时候，Scala 能够推导出方法的返回类型。假如方法不接受任何参数，你还可以在方法定义中省略参数列表。

使用等号也强调了函数式编程的一个准则：值和函数是高度对齐的概念。正如我们所看到的那样，函数可以作为参数传递给其他函数，也能够返回函数，还能被赋给某一变量。这与对象的行为是一致的。

最后提一下，假如方法体仅包含一个表达式，那么Scala 允许你省略花括号。所以说，使用等号能够避免可能的解析歧义。

## Scala 中尽量少使用return关键字
在Scala 中，函数或方法中把最后一条表达式的返回值作为自己的返回值。尽管Scala 中存在return 关键字，但只能在方法中使用，**匿名函数中则不允许使用。事实上，方法中也很少用到这个关键字。**

## 伴生对象的定义
你可以自己定义伴生对象。任何时候只要对象名和类名相同并且定义在同一个文件中，这些对象就能称作伴生对象。

任何时候只要你在输入对象后紧接着输入一个参数列表，Scala 就会查找并调用该对象的apply 方法，这也意味着下面两行代码是等价的。
```scala
val p1 = Point.apply(1.0, 2.0)
val p2 = Point(1.0, 2.0)
```

紧挨着对象名输入参数列表时，Scala 会查找并调用匹配该参数列表的apply方法。换句话说，Scala 会猜想该对象定义了apply 方法。从句法角度上说，任何包含了apply 方法的对象的行为都很像函数。

在伴生对象中安置apply 方法是Scala 为相关类定义工厂方法的一个便利写法。

**在类中定义而不是在对象中定义的apply 方法适用于该类的实例。例如，调用Seq.apply(index:Int) 方法将获得序列中指定位置的元素（从0开始计数），这里的Seq是一个类的实例对象。**

## 分号
分号是表达式之间的分隔符,可以推断得出。当一行结束时,Scala 就认为表达式结束了,除非它可以推断出该表达式尚未结束,应该延续到下一行,如下面这个例子:

```scala
// Trailing equals sign indicates more code on the next line.
// 末尾的等号表明下一行还有未结束的代码。
def equalsign(s: String) =
  println("equalsign: " + s)

// Trailing opening curly brace indicates more code on the next line.
// 末尾的花括号表明下一行还有未结束的代码。
def equalsign2(s: String) = {
  println("equalsign2: " + s)
}

// Trailing commas, periods, and operators indicate more code on the next line.
// 末尾的逗号、句号和操作符都可以表明,下一行还有未结束的代码。
def commas(s1: String,
           s2: String) = Console.
  println("comma: " + s1 + 
          ", " + s2)
```

与编译器相比,REPL 更容易将每行视为单独的表达式。因此,在 REPL 中输入跨越多行的表达式时,最安全的做法是每行(除最后一行外)都以上述脚本中出现过的符号结尾。反过来,你可以将多个表达式放在同一行中,表达式之间用分号隔开。

如果你需要将多行代码解释为同一表达式,却被系统视为多个表达式,**可以使用 REPL 的 :paste 模式。输入 :paste,然后输入你的代码,最后用Ctrl-D 结束。**

## var和val 变量声明
在声明变量时,Scala 允许你决定该变量是不可变(只读)的,还是可变的(读写)。

var 和 val 关键字只标识引用本身是否可以指向另一个不同的对象,它们并未表明其所引用的对象是否可变。

## Scala实现集合中相邻元素间的差值
**概要**

计算同一集合中元素两两之间的差值，即求开始集合(a,b,c,d)中(b-a,c-b,d-c)

**解法**

```scala
val l1 = 1 to 10 toList
val l2 = l1.tail
l1.zip(l2).map(p=>(p._2 - p._1)
```
上述代码即可求出两两差值。代码含义稍作解释:

* tail表示一个集合中除首元素外的剩余元素，也是一个集合。  
* zip将两个集合组成一个数值对集合，比如原来有两个集合分别是(a,b,c,d), (1,2,3,4)，使用zip之后，形成((a,1),(b,2),(c,3),(d,4))  
* map操作中的p._1和p._1表示tuple中的第一个元素和第二个元素

**衍生**

既然讲了首元素和剩余元素，肯定要讲到其对立面。分别引述如下

* **head|tail**  head表示头元素，tail表示除head之外的剩余元素集合  
* **init|last**    last表示尾元素, init表示除尾元素之外的剩余元素集合  

## Scala传名参数
Map的方法其实就是一个传名参数：
```scala
  /**  Returns the value associated with a key, or a default value if the key is not contained in the map.
   *   @param   key      the key.
   *   @param   default  a computation that yields a default value in case no binding for `key` is
   *                     found in the map.
   *   @tparam  B1       the result type of the default computation.
   *   @return  the value associated with `key` if it exists,
   *            otherwise the result of the `default` computation.
   *
   *   @usecase def getOrElse(key: A, default: => B): B
   *     @inheritdoc
   */
  def getOrElse[B1 >: B](key: A, default: => B1): B1 = get(key) match {
    case Some(v) => v
    case None => default
  }
```

传名参数 仅在被使用时触发实际参数的求值运算。 它们与 传值参数 正好相反。 要将一个参数变为传名参数，只需在它的类型前加上 =>。
```scala
def calculate(input: => Int) = input * 37
```
**传名参数的优点是，如果它们在函数体中未被使用，则不会对它们进行求值。而传值参数的优点是它们仅被计算一次。** 以下是我们如何实现一个 while 循环的例子：
```scala
def whileLoop(condition: => Boolean)(body: => Unit): Unit =
  if (condition) {
    body
    whileLoop(condition)(body)
  }

var i = 2

whileLoop (i > 0) {
  println(i)
  i -= 1
}  // prints 2 1
```
方法 whileLoop 使用多个参数列表来分别获取循环条件和循环体。 如果 condition 为 true，则执行 body，然后对 whileLoop 进行递归调用。 如果 condition 为 false，则永远不会计算 body，因为我们在 body 的类型前加上了 =>。

现在当我们传递 i > 0 作为我们的 condition 并且 println(i); i-= 1 作为 body 时，它表现得像许多语言中的标准 while 循环。

如果参数是计算密集型或长时间运行的代码块，如获取 URL，这种延迟计算参数直到它被使用时才计算的能力可以帮助提高性能。

## 应该广泛使用case语句偏函数语法简化函数字面量的实现
使用case定义函数字面量可以将参数列表声明与嵌套变量的提取合二为一，简化函数的实现。

不但需要单个参数的单个参数可以用偏函数，多个参数也可以用偏函数，把所有的参数当成一个元组就行，这可能是编译器提我们做好的，反正是可以用的，很方便。
```scala
val list = Seq((1,2),(3,4))
list.map{case (num1, num2) => (num2 + num1) * (num2 - num1)}
//Seq[Int] = List(3, 7)

//测试多个参数的情况
def test(x:Int, y:Int)( f:(Int, Int) => Int ):Int =  f(x,y)
test(2, 3){case (x,y) => x + y}
//Int = 5

//参数更复杂的情况
def test2(x:(Int, Int), y:(Int, Int))( f:((Int, Int), (Int, Int)) => (Int, Int) ):(Int, Int) =  f(x,y)
test2((1, 2), (3, 4)){case ((x1,y1), (x2,y2)) => (x1+x2,y1+y2)}
//(Int, Int) = (4, 6)

//函数参数不是一个独立的参数列表的情况
def test3(x:Int, y:Int, f:(Int, Int) => Int ):Int =  f(x,y)
test3(2, 3, {case (x,y) => x + y})
//Int = 5

//函数参数不是一个独立的参数列表的情况
def test4(x:(Int, Int), y:(Int, Int), f:((Int, Int), (Int, Int)) => (Int, Int) ):(Int, Int) =  f(x,y)
test4((1, 2), (3, 4), {case ((x1,y1), (x2,y2)) => (x1+x2,y1+y2)})
//(Int, Int) = (4, 6)
```

## 推断类型信息
静态类型语言的代码往往比较繁琐。因此我们可以考虑使用以下 Java 中的类型声明代码(Java 7 之前的版本):
```java
import java.util.HashMap;
...
HashMap<Integer, String> intToStringMap = new HashMap<Integer, String>();
```

Java 7 引入了尖括号操作符来推断表达式右边的泛型类型,降低了冗余度:
```java
HashMap<Integer, String> intToStringMap = new HashMap<>();
```

利用自动推断类型信息,以上声明可以用 Scala 重写如下:
```scala
val intToStringMap: HashMap[Integer, String] = new HashMap
```

**如果我们将 HashMap[Integer, String] 放在等号后边,代码会更简洁**:
```scala
val intToStringMap2 = new HashMap[Integer, String]
```

一些函数式编程语言,如 Haskell,可以推断出几乎所有的类型,因为它们可以执行全局类型推断。Scala 则无法做到这一点,部分原因是 Scala 必须支持子类多态(支持继承),这使得类型推断要困难得多。

**以下总结了在 Scala 中什么时候需要显式类型注解。**在实际编程中,你在以下情况中必须提供显式的类型注解:  
* 声明了可变的 var 变量或不可变的 val 变量,没有进行初始化。(例如,在类中的抽象声明,如 val book: String, var count: Int )。  
* 所有的方法参数(如 def deposit(amount: Money) = {...} )。  
* 方法的返回值类型,在以下情况中必须显式声明其类型。  
**– 在方法中明显地使用了 return (即使在方法末尾也是如此)**  
**– 递归方法。**  
**– 两个或多个方法重载(拥有相同的函数名),其中一个方法调用了另一个重载方法,调用者需要显式类型注解。**  
**– Scala 推断出的类型比你期望的类型更为宽泛,如 Any 。**  

## 一些 Java 中的方法名在 Scala 中是保留字问题
一些 Java 中的方法名在 Scala 中是保留字。如 java.util.Scanner.match 。为了避免编译错误,引用该方法名时,在名字两边加上反引号,如 java.util.Scanner.`match` 。

Scala 保留字中没有 break 和 continue 。这两个流程控制的关键字在 Scala 中不存在。Scala 鼓励使用函数式编程的惯用法来实现相同的 break 、 continue 功能。函数式编程通常会更加简洁,不容易出现 bug。

## 函数字面量
(i: Int, s: String) => s+i 是一个类型为 Function2[Int,String,String] (返回值类型为 String )的函数字面量。

以下三种声明是等价的:
```scala
val f = (i: Int, s: String) => s+i
f: (Int, String) => String = <function2>

val f1: (Int,String) => String = (i, s) => s+i
f1: (Int, String) => String = <function2>

val f2: Function2[Int,String,String] = (i, s) => s+i
f2: (Int, String) => String = <function2>
```

## 封闭类(密封类)的继承
现在我们来探讨 Option 的一个很有用的特性。 Option 的一个关键点在于它只有两个有效的子类。如果我们有值,则对应 Some 子类;如果没有值,则对应 None 子类。没有其他有效的 Option 子类型。所以,我们可以防止用户创建一个他们自己的子类。

为了达到这个目的,Scala 设计了关键字 sealed 。 Option 的声明类似这样(省略部分细节):
```scala
sealed abstract class Option[+A] ... { ... }
```

关键字 sealed 告诉编译器,所有的子类必须在同一个源文件中声明。而在 Scala 库中,Some 与 None 就是与 Option 声明在同一源文件中的。这一技术有效防止了 Option 派生其他子类型。

顺便提一下,如果需要防止用户派生任何子类,也可以用 final 关键字进行声明。

## 导入类型及其成员
就像在 Java 中一样,要使用包中的声明,必须先导入它们。但 Scala 还提供了其他选择,以下例子展示了 Scala 如何导入 Java 类型:
```scala
import java.awt._
import java.io.File
import java.io.File._
import java.util.{Map, HashMap}
```

你可以像第一行那样,用下划线( _ )当通配符,导入包中的所有类型。你也可以像第二行那样导入包中单独的 Scala 类型或 Java 类型。

**`Java 用星号(*)作为 import 的通配符。在 Scala 中,星号被允许用作函数名,因此 _ 被用作通配符,以避免歧义。`**例如,如果对象 Foo 定义了其他方法,同时它还定义了 * 方法,import Foo.* 表示什么呢?

第三行导入了 java.io.File 中所有的静态方法和属性。与之等价的 Java import 语句为import static java.io.File.* 。Scala 没 有 import static 这 样 的 写 法, 因 为 Scala 将object 类型与其他类型一视同仁。

如第四行所示,选择性导入的语法非常好用,在第四行中我们导入了 java.util.Map 和java.util.HashMap 。

import 语句几乎可以放在任何位置上,所以你可以将其可见性限制在需要的作用域中,可以在导入时对类型做重命名,也可以限制不想要的类型的可见性:
```scala
def stuffWithBigInteger() = {
    import java.math.BigInteger.{
        ONE => _,
        TEN,
        ZERO => JAVAZERO }
    
    // println( "ONE: "+ONE )  // ONE未定义
    println( "TEN: "+TEN )
    println( "ZERO: "+JAVAZERO )
}
```

由于这一 import 语句位于 stuffWithBigInteger 函数中,导入的类型和值在函数外是不可见的。

将 java.math.BigInteger.ONE 常量重命名为下划线,使得该常量不可见。当你需要导入除少部分以外的所有声明时,可以采用这一技术。

接着, java.math.BigInteger.TEN 导入时未经重命名,所以可以用 TEN 引用它。

最后, java.math.BigInteger.ZERO 常量被赋予了 JAVAZERO 的“别名”。

当你想取一个便利的名字或避免与当前作用域中其他同名声明冲突时,别名非常有用。**导入 Java 类型时经常使用别名,以避免其余 Scala 中同名类型的冲突**,如 java.util.List 与 java.util.Map ,在 Scala 库中有相同名称的类。

## 抽象类型与参数化类型
Scala 支持参数化类型,与 Java 中的泛型十分类似。(我们也可以交换这两个术语,但 Scala 社区中多使用“参数化类型”,Java 社区中常用泛型一词。)在语法上,Java 使用尖括号( <...> ),而 Scala 使用方括号( [...] ),因为在 Scala 中 < 和 > 常用作方法名。

Scala 还支持另一种被称为“抽象类型”的抽象机制,它可以运用在许多参数化类型中,也能够解决设计上的问题。然而,尽管两种机制有所重合,但并不冗余,两种机制对不同的设计问题各有优势与不足。

参数化类型和抽象类型都被声明为其他类型的成员,就像是该类型的方法与属性一样。以下示例在父类中使用抽象类型,而在子类中将该类型具体化:
```scala
// src/main/scala/progscala2/typelessdomore/abstract-types.sc
import java.io._

abstract class BulkReader {
  type In
  val source: In
  def read: String  // Read source and return a String
}

class StringBulkReader(val source: String) extends BulkReader {
  type In = String
  def read: String = source
}

class FileBulkReader(val source: File) extends BulkReader {
  type In = File
  def read: String = {
    val in = new BufferedInputStream(new FileInputStream(source))
    val numBytes = in.available()
    val bytes = new Array[Byte](numBytes)
    in.read(bytes, 0, numBytes)
    new String(bytes)
  }
}

println(new StringBulkReader("Hello Scala!").read)
// Assumes the current directory is src/main/scala:
println(new FileBulkReader(
  new File("TypeLessDoMore/abstract-types.sc")).read)
```

抽象类 BulkReader 声明了 3 个虚拟成员:一个名为 In ,是类型成员;第二个类型为 In ,是 val 变量,名为 source ;第三个是一个 read 方法。

派生类 StringBulkReader 与 FileBulkReader 为上述抽象成员提供具体化的定义。

注意 type 成员的工作机制与参数化类型中的类型参数非常类似。事实上,我们可以将该示例重写如下,在这里我们只显示改动的部分:
```scala
abstract class BulkReader[In] {
  val source: In
  ...
}
class StringBulkReader(val source: String) extends BulkReader[String] {...}
class FileBulkReader(val source: File) extends BulkReader[File] {...}
```

就 像 参 数 化 类 型, 如 果 我 们 定 义 In 类 型 为 String , 则 source 属 性 也 必 须 被 定 义 为String 。注意 StringBulkReader 的 read 方法只是将 source 属性返回,而 FileBulkReader的 read 方法则需要读取文件的内容。

那么,类型成员与参数化类型相比有什么优势呢?当类型参数与参数化的类型无关时,参数化类型更适用。例如 List[A] , A 可能是 Int 、 String 或 Person 等。而当类型成员与所封装的类型同步变化时,类型成员最适用。正如 BulkReader 这个例子,类型成员需要与封装的类型行为一致。有时这种特点被称为家族多态,或者协特化。

## 操作符重载 
在 Scala 中,几乎所有的“操作符”其实都是方法。我们一起看看最基础的一个例子:
```scala
1 + 2
```
数字之间的加号是什么呢?这个操作符是一个方法。

首先,请注意在 Scala 的世界里,Java 中特殊的“基本类型”都变成了正规的对象,这些对象类型为: Float 、 Double 、 Int 、 Long 、 Short 、 Byte 和 Boolean 类型。这也意味着它们可以拥有成员方法。

**所有的操作符都是方法。假如你知道该知识点,便能够更容易的理解 Scala 代码。**有时候你会看到一些新的操作符,不过你无需担心那些特殊的情况(这些新的操作符本质都是方法,所以无需担心)。actor 会互相发送异步消息,发送消息时使用了感叹号 ! 操作符,这一操作符只是一个普通方法罢了。

这种灵活的命名方式让编写出的类库非常自然,就像 Scala 自身的一种延伸。利用这种命名方式,你可以编写一个新的数学库,其中的数值类型支持所有的数学操作。你也可以编写一个与 actor 行为类似的全新的并发消息处理层。除了少数的一些命名规则限制之外,使用这些命名方式能创造出无限的可能。

**能够创建操作符符号并不意味着你应该这样做。**当你定义 API 时,要提醒自己用户很难读懂这些隐晦的标点符号式的操作符,更别提学会和记住了。**滥用这些操作符只会使你的代码变得晦涩。**因此,如果你沉迷于创建新的操作符,**一旦该操作符无法带来便利,那就意味着你凭空牺牲了方法命名的可读性。**

## 无参数方法
我们在定义无参方法时可以省略括号。一旦定义无参方法时省略了括号,那么在调用这些方法时必须省略括号。与之相反,假如在定义无参方法时添加了空括号,那么调用方可以选择省略或是保留括号。

例如, List.size 的方法定义体中省略了括号,因此你应该编写 List(1,2,3).size 这样的代码。假如你尝试输入 List(1,2,3).size() ,系统将会返回错误。

java.lang.String 的 length 方法定义体中则包含了括号(这是为了能在 Java 中运行),而Scala 同时支持 "hello".length() 和 "hello".length 这两种写法。这同样适用于 Scala 定义的一些定义体中包含空括号的那些无参方法。

为了实现与 Java 语言的互操作,无参方法定义体中出现了是否包含空括号这两种情况的处理规则之间的不一致性。尽管 Scala 也希望定义和使用保持一致(如果定义体中包含括号,那么调用时必须添加括号,反之,调用时必须省略括号),不过由于包含了空括号的定义会更灵活些 ,这确保了调用 Java 无参方法时可以与调用 Scala 无参方法保持一致。

Scala 社区已经养成了这样一个习惯: 定义那些无副作用的无参方法时省略括号,例如:集合的 size 方法。定义具有副作用(例如,方法中会有 I/O 操作)的方法时则添加括号,这样便能提醒读者某些对象可能会发生变化,需要额外小心。

## extends和with的使用
总结起来就是这样：

* 一个类只能继承与一个类，但可以实现和混入多个特质。trait类似于java8中的接口。  
* 类继承的时候第一个关键字使用extends，之后的都使用with   
* 新建对象混入trait时只能使用with，可以混合多个trait   

看下面用例
```scala
trait Mytrait1{
  def doSomething(str:String):Unit={
    println(s"$str is doing")
  }
}
trait Mytrait2{
  def forfun():Unit
}
class Myclass1 extends Mytrait1{

}

class Myclass2{}
class Myclass3 extends Mytrait1 with Mytrait2{
  override def forfun(): Unit = ???
}

val test = new Myclass2 with Mytrait2{
  @Override
  override def forfun(): Unit = {
    println("666")
  }
}
val test2 = new Myclass1

test forfun()
test2 doSomething("Test")
```
这里我们声明了特质Mytrait1，Mytrait2，和Myclass（1~3）这里Myclass继承了Mytrait1的特质，用了extends关键字，Myclass3继承了Mytrait1，和Mytrait2特质，第一个trait之前用了extends而第二个trait之前用了with，而实例化Myclass的时候在trait之前又用了with关键字。 

这里特意声明一下: **如果是待声明的类没有扩展其他类只是或是实现了一些trait那么必须用extends关键字，并把它用在第一个trait之前而其它的trait之前用with关键字，就像Myclass3， 如果在实例化某一个类型的时候想要混入某一个特质，这个时候要用with关键字，如new Mycalss2这一个声明一样。**

## 不带参数的抽象方法可以在子类中用 val 变量实现
在父类型中,不带参数的抽象方法可以在子类中用 val 变量实现。推荐的做法是:在抽象父类型中声明一个不带参数的抽象方法,这样就给子类型如何具体实现该方法留下了巨大的自由,既可以用方法实现,也可以用 val 变量实现。

一个抽象的,不带参数的父类方法,在子类型中可以用一个 val 实现。这是由于 val 的值是固定的(必定的),而一个不带参数、返回值为某类型变量的方法可以返回任意一个该类型的变量。这样,使用 val 实现的方法在返回值上严格符合方法定义,当方法被“调用”时,使用 val 变量与真实调用方法一样安全。事实上,这是透明引用的一个应用。在透明引用中,我们用一个值代替一个总是返回固定值的表达式!

## 单例的类如何表示？
```scala
object A

val a : A.type = A
```
其实对任何实例x都存在一个x.type这样的类型。

单例类型是个特殊的类型，单例类型绑定(依赖)在某个对象实例上，每个对象实例都有它的单例类型。不过它的场景并不多见。

看一个绕开JVM类型擦除的例子(里面涉及到隐式值和单例类型)：
```scala
// src/main/scala/progscala2/implicits/implicit-erasure.sc
object M {
  implicit object IntMarker                                          // <1>
  implicit object StringMarker

  def m(seq: Seq[Int])(implicit i: IntMarker.type): Unit =           // <2>
    println(s"Seq[Int]: $seq")

  def m(seq: Seq[String])(implicit s: StringMarker.type): Unit =     // <3>
    println(s"Seq[String]: $seq")
}

import M._                                                           // <4>
m(List(1,2,3))
m(List("one", "two", "three"))
```
➊ 上面的代码中定义了两个具有特殊用途的隐式对象,这两个对象将用于解决由于类型擦除导致的方法二义性问题。  
➋ 重新定义输入参数为 Seq[Int] 类型的方法。现在该方法新增了第二个参数列表,新增 的 参 数 列 表 希 望 能 够 接 收 到 一 个 隐 式 IntMarker 对 象。 请 注 意 该 对 象 的 类 型 是IntMarker.type 。该类型引用了单例对象的类型!  
➌ 重新定义输入参数为 Seq[String] 的方法。  
➍ 导入并使用隐式值和方法。这些代码能够顺利通过编译并打印出正确的输出。  

## Predef默认导入的Seq类型
为了鼓励程序员使用不可变的集合类型, Predef 及 Predef 中使用的其他类型在暴露部分不可变集合类型时,不需要显式导入或使用全路径导入。如: List 和 Map 。

在上述规则中,Scala 只暴露了不可变集合类型,然而, Predef 还将 scala.collection.Seq导入到了当前作用域。 scala.collection.Seq 中的类型是可变集合类型和不可变集合类型共同的抽象类型。

尽 管 存 在 scala.collection.immutable.Seq ( scala.collection.Seq 的 一 个 子 类 型 ), 但 Predef 导 入 的 是 scala.collection.Seq 而 非 scala.collection.immutable.Seq , 主 要 原因是这方便处理 Java 的 Array 。如同 Seq 一样, Array 和其他集合类型有着相同的处理方式。Java 的 Array 是可变集合类型,Scala 的其他可变集合类型也大部分都实现了 scala.collection.Seq 。

这样一来, scala.collection.Seq 虽然没有暴露任何用于修改集合的方法,但仍存在并发情况下出错的潜在风险,因为可变集合类型不是线程安全的,因此必须对它做特殊处理。

假设并发库中的方法将 Seq 作为参数,但你只希望传入不可变集合类型。此时 Seq 的使用就产生了一个漏洞,因为客户端可以传入可变集合类型,如 Array 。

**应该记住,Seq 默认的实际类型为 scala.collection.Seq。因此,传入的Seq 类型的实例可能是可变的,所以线程是不安全的。**

**Scala 计 划 在 2.12 版 本( 即 下 一 个 发 行 版 ) 中 将 scala.Seq 改 为 scala.collection.immutable.Seq 的指向别名。**

## scala 和 java 的协变、逆变、不变(非转化) 比较
Scala 参数化类型和 Java 参数化类型(在 Java 中,通常称为泛型,generic)的一个重要区别在于,继承差异机制如何工作。

假设一个方法带有的参数类型为 List[AnyRef] ,你可以传入 List[String] 吗?换句话说,List[String] 是否应该被看作是 List[AnyRef] 的一个子类型呢?如果是,这种转化称为协变(covariance)。因为容器(被参数化的类型)的继承关系与参数类型的继承关系的“方向一致”。

同样存在类型是逆变(contravariant)的,对于特定类型 X , X[String] 是 X[Any] 的父类。

如果参数化类型既不是协变的,也不是逆变的,我们称之为非转化(invariant)的。相反地,有的参数化类型可以同时拥有两种或两种以上的这类属性。

Java 和 Scala 均支持协变,逆变和非转化类型。然而,在 Scala 中,转化行为的定义是类型声明的一部分,称为转化标记(variance annotation)。我们使用 + 来表示协变类型;使用 -表示逆变类型;非转化类型不需要添加标记。换句话说,类型的设计者决定该类型在继承体系中如何进行转化。

以下是几个声明示例:
```scala
class W[+A] {...}       //协变
class X[-A] {...}       //逆变
class Y[A] {...}        //非转化
class Z[-A,B,+C] {...}  //混合
```

相反,Java 中参数化的类型在定义时并未声明继承转化行为,而是在使用该类型时,也就是在声明变量时,才指定参数化类型的转化行为。

下表总结了 Java 和 Scala 中的三种转化标记及其意义。其中 Tsup 是 T 的父类,而 Tsub是 T 的子类。

| Scala | Java 描述   |                                                      |
| ----- | ----------- | ---------------------------------------------------- |
| +T    | ? extends T | 协变(如 List[T sub ] 是 List[T] 的子类)              |
| -T    | ? super T   | 逆变(如 X[T sup ] 是 X[T] 的子类)                    |
| T     | T           | 非转化继承(不能用 Y[T sup ] 或 Y[T sub ] 代替 Y[T] ) |

回 到 List 的 讨 论,Scala 的 List 实 际 上 被 声 明 为 List[+A] , 意 味 着 List[String] 是List[AnyRef] 的子类,所以对于类型参数 A , List 是协变的。当 List 只有一个协变的类型参数时,你会经常听到一种简称,即“列表是协变的”。相应地,对于逆变类型也有类似的叫法。

## 可变类型的变异(注意scala中可变集合是不可变的)
可变类型只允许非变异行为。考虑以下示例:
```scala
//编译报错
class ContainerPlus[+A](var value: A)
//编译报错
class ContainerMinus[-A](var value: A)
```

对于 getter 和 setter 方法中的可变字段而言,它在读方法中处于协变的位置,而在写方法中又处于逆变的位置。不存在既协变又逆变的类型参数,所以对于可变字段 A 的唯一选择就是非变异。

**注意：scala中不可变集合一部分是协变的，可变集合都是是不可变的。**

scala中不可变集合的斜边情况：
```scala
//序列基本全都是协变的
//scala包对象中定义
type Seq[+A] = scala.collection.Seq[A]
type List[+A] = scala.collection.immutable.List[A]

//Map的key是不变的，value是协变的，Set是不变的。
//Set是不变的应该和set继承(A=>Boolean)需要实现def apply(e: A): Boolean方法有关吧，Function的参数是逆变的，所以Set只能是不变的。
//Predef对象中定义
type Map[A, +B] = immutable.Map[A, B]
type Set[A]     = immutable.Set[A]
type Function[-A, +B] = Function1[A, B]
```


scala中函数的参数是协变的，返回值是逆变的(了解即可，这些东西相同的话比较绕)。看一下scala.Function2 的声明:
```scala
trait Function2[-T1, -T2, +R] extends AnyRef
```

## ???函数
```scala
def ???: Nothing
```
在一个尚未实现的方法的方法体中调用。它为方法提供了具体的定义,允许编译器将方法所属的类型视为具体(与抽象对应)的类。然而,如果调用该方法,就会抛出 scala.NotImplementedError 异常。

我们可以临时调用 ??? 方法来定义其他的方法,使得方法定义完整,并通过编译。但如果调用该方法,就会抛出异常。以下是 ??? 的定义:
```scala
package scala
object Predef {
  ...
  def ??? : Nothing = throw new NotImplementedError
  ...
}
```

In [1]:
val list = List((1,2),3,4)

list: List[Any] = List((1,2), 3, 4)

In [2]:
for ( (k,v) <- list) print(k,v)

(1,2)

In [6]:
for ( x @ 3 <- list) print(x)

3